In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delhi-dtaset/stn_coord.csv
/kaggle/input/delhi-dtaset/DelhiData_3hr_sample_Combine.csv


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import torch.nn.functional as F
import torch.optim as optim
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler
from geopy.distance import geodesic

In [3]:
def summarize_tensor(name, tensor):
    print(f"{name}: shape={tuple(tensor.shape)}, min={tensor.min().item():.4f}, max={tensor.max().item():.4f}, mean={tensor.mean().item():.4f}, has_nan={torch.isnan(tensor).any().item()}")

# Node and Edge MLP

In [4]:
class EdgeMLP(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim, edge_feature_dim):
        super(EdgeMLP, self).__init__()
        self.MLP = nn.Sequential(
            nn.Linear(7, hidden_dim),  #concat 2 node feature 2 advec_coefficient
            nn.ReLU(),
            nn.Linear(hidden_dim, edge_feature_dim)
        )

    
    def forward(self, h_j, h_i, P_ji):
        # print("h_j:", h_j.shape)  
        # print("h_i:", h_i.shape)  

        # P_ji_resized = P_ji.repeat(256 // 16, 250, 1, 1) 
        # P_ji_resized = P_ji_resized[:256, :, :, :] 
        # P_ji_resized = P_ji_resized[:, :50, :, :] 

        # summarize_tensor('h_i',h_i)
        # summarize_tensor('h_j',h_j)
        B, N, N, _ = h_j.shape
        P_ji_resized = P_ji.reshape(-1, N, N)         
        P_ji_resized = P_ji_resized.unsqueeze(-1)       
        # summarize_tensor('P_ji_resized',P_ji_resized)
        # print("P_ji:", P_ji_resized.shape)  
        
        x = torch.cat([h_j, h_i, P_ji_resized], dim=-1)
        # summarize_tensor('x cat',x)

        del P_ji_resized
        return self.MLP(x)

In [5]:
class NodeMLP(nn.Module):
    def __init__(self, edge_feature_dim, hidden_dim, node_feature_dim):
        super(NodeMLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(edge_feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, node_feature_dim)  # OUTPUT 25 features
        )

    def forward(self, edge_features, adj):
        # edge_features: (B*T, N, N, edge_feature_dim)
        B, N, _, _ = edge_features.shape
        edge_features = edge_features.view(B, N*N, -1)
        out = self.mlp(edge_features)
        out = out.view(B, N, N, -1)
        out = out.sum(dim=2)  # Sum over neighbors

        del edge_features
        return out

In [6]:
def advection_coefficient(v, D_ji, theta):
    # v, theta : (B, 16, N)
    # D_ji : (N, N)

    B, T, N = v.shape       # batch, timesteps, nodes
    
    D_expanded   = D_ji.unsqueeze(0).unsqueeze(0).expand(B, T, N, N)
    
    v_expanded    = v    .unsqueeze(-1).expand(B, T, N, N)
    theta_expanded= theta.unsqueeze(-1).expand(B, T, N, N)

    D_ji_expanded = torch.clamp(D_ji,1e-3)
    
    P_ji = F.relu(torch.abs(v_expanded) * torch.cos(theta_expanded) /( D_ji_expanded))

    del v_expanded, theta_expanded, D_expanded
    return P_ji

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# GAT layer

In [8]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.6, alpha=0.2):
        super(GATLayer, self).__init__()
        self.dropout = dropout
        self.alpha = alpha
        
        self.W = nn.Linear(in_features, out_features)
        self.a = nn.Linear(2 * out_features, 1)
        
        self.leakyrelu = nn.LeakyReLU(alpha)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W.weight)
        nn.init.xavier_uniform_(self.a.weight)

    def forward(self, h, adj):
        """
        Args:
            h: Input features [B, N, F]
            adj: Adjacency matrix [B, N, N]
        """
        # print("h shape:",h.shape)
        B, N, _ = h.shape
        
        # Linear transformation
        Wh = self.W(h)  # [B, N, F']

        
        # Create node pairs
        Wh_1 = Wh.unsqueeze(2)  # [B, N, 1, F']
        Wh_2 = Wh.unsqueeze(1)  # [B, 1, N, F']
        
        # Concatenate all pairs
        Wh_pairs = torch.cat([Wh_1.expand(-1, -1, N, -1), 
                             Wh_2.expand(-1, N, -1, -1)], dim=-1)  # [B, N, N, 2F']
        
        # Compute attention scores
        e = self.a(Wh_pairs).squeeze(-1)  # [B, N, N]
        e = self.leakyrelu(e)
        
        
        # Normalize attention scores
        attention = F.softmax(e, dim=-1)  # [B, N, N]
        attention = F.dropout(attention, self.dropout, training=self.training)
        
        h_prime = torch.bmm(attention, Wh)  # [B, N, F']

        del Wh, Wh_1, Wh_2, Wh_pairs, e,attention
        return F.relu(h_prime, inplace=True)

# ClusterSpatial

In [9]:

class ClusterSpatial(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_clusters=3):
        super(ClusterSpatial, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_clusters = n_clusters

        # One GRU per cluster
        self.grus = nn.ModuleList([
            nn.GRU(input_dim, hidden_dim, batch_first=True)
            for _ in range(n_clusters)
        ])

        # Final linear layer to predict PM2.5
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, H_t, cluster_indices):
        """
        H_t: Tensor of shape (B, N, input_dim)
        cluster_indices: 1D Tensor or array of length N with values in [0..n_clusters-1]
        """
        # Ensure cluster_indices is a tensor on the same device
        # if not isinstance(cluster_indices, torch.Tensor):
        #     cluster_indices = torch.tensor(cluster_indices, device=H_t.device)
        # else:
        #     cluster_indices = cluster_indices.to(H_t.device)

        B, N, _ = H_t.size()
        device = H_t.device

        # Prepare the output tensor for all nodes
        h_t_out = torch.zeros(B, N, self.hidden_dim, device=device)

        # Process each cluster separately
        for k in range(self.n_clusters):
            # Create mask for nodes in cluster k
            mask = (cluster_indices == k)
            if not mask.any():
                continue  

            # print("H_t shape",H_t.shape)
            # print("mask shape",mask.shape)
            # (B, num_k, input_dim)
            H_k = H_t[:, mask, :]

            # GRU -> outputs over sequence dimension (here node dimension)
            gru_out, _ = self.grus[k](H_k)
            # Use the last time step's output
            out_k = gru_out[:, -1, :]

            out_expanded = out_k.unsqueeze(1).expand(-1, mask.sum().item(), -1)

            h_t_out[:, mask, :] = out_expanded

        pm25_pred = self.fc(h_t_out).squeeze(-1)  # shape (B, N)
        return pm25_pred


# Spatiotemporal

In [10]:
class Spatiotemporal(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim, gat_out_dim, num_clusters):
        super(Spatiotemporal, self).__init__()

        self.edge_MLP = EdgeMLP(node_feature_dim, hidden_dim, edge_feature_dim)  # eq 3
        self.node_MLP = NodeMLP(edge_feature_dim, hidden_dim, node_feature_dim)  # eq 4
        self.gat = GATLayer(node_feature_dim, gat_out_dim)  # eq 5-7
        self.cluster = ClusterSpatial(node_feature_dim * 2 + gat_out_dim, hidden_dim, num_clusters)  # eq 8-10

    def forward(self, h_t, v, D_ji, adj, theta, cluster_indices, device):
        h_t = h_t.to(device)
        v = v.to(device)
        D_ji = D_ji.to(device)
        theta = theta.to(device)
        adj = adj.to(device)
        # cluster_indices = cluster_indices.to(device)

        B, T, N, F = h_t.shape  # B: Batch size, T: Time steps, N: Nodes, F: Features
        h_t = h_t.reshape(B * T, N, F)  # (B*T, N, F)
        
        # summarize_tensor("h_t", h_t)
        
        # advection coefficient P_t_ji (pollutant transport feature)
        P_ji = advection_coefficient(v, D_ji, theta)  # eq 2
        P_ji = P_ji.to(device)
        # summarize_tensor("P_ji", P_ji)

        #  h_i and h_j to calculate edge features
        h_i = h_t.unsqueeze(2).expand(-1, -1, N, -1)
        h_j = h_t.unsqueeze(1).expand(-1, N, -1, -1)

        # Compute edge features using h_i, h_j and P_ji
        edge_features = self.edge_MLP(h_j, h_i, P_ji)  # eq 3
        # summarize_tensor("edge feature ", edge_features)

        # Aggregate edge features into node features
        zeta = self.node_MLP(edge_features, adj)  # eq 4
        
        adj = adj.unsqueeze(1)          # (B, 1, N, N)
        adj = adj.expand(-1, T, -1, -1) # (B, T, N, N)

        # Flatten adjacency and features for GAT layer
        adj_flat = adj.reshape(B * T, N, N)
        h_t_flat = h_t.reshape(B * T, N, F)


        # Compute GAT attention outputs
        h_attn_flat = self.gat(h_t_flat, adj_flat)  # eq 5-7

        h_attn = h_attn_flat

        # print("h_t shape:", h_t.shape)
        # print("h_attn shape:", h_attn.shape)
        # print("zeta shape:", zeta.shape)

        H_i = torch.cat([h_attn, zeta, h_t], dim=-1)  # eq 8
        H_i = H_i.view(B * T, N, -1)  # Flatten B*T
        # summarize_tensor("H_i",H_i)
        
        outputs = self.cluster(H_i, cluster_indices)  # eq 9-10
        outputs = outputs.view(B, T, N, -1)                 # (B, T, N, num_clusters)
        # summarize_tensor("outputs",outputs)

        del P_ji, h_i, h_j, edge_features, zeta, adj_flat, h_t_flat, h_attn_flat, h_attn, H_i 
        return outputs

# Temporal

In [11]:
class Temporal_Module(nn.Module):
    def __init__(self, num_nodes, cnn_channels, kernel_size, gru_hidden_dim, future_step, nodes):
        super(Temporal_Module, self).__init__()

        self.future_step = future_step
        
        # 1D CNN extracts the historical feature trends
        self.CNN_1D = nn.Conv1d(in_channels = 3,# 3 feature de rhe pm2.5 theta aur v
                                out_channels = cnn_channels,
                                kernel_size = kernel_size)

        # Applies a GRU to model sequential dependencies from the CNN features over time
        self.gru = nn.GRU(input_size = cnn_channels,
                          hidden_size = gru_hidden_dim,
                          num_layers = 2, 
                          batch_first = True)

        # MLP for temporal features from GRU output
        self.MLP = nn.Sequential(
            nn.Linear(gru_hidden_dim, gru_hidden_dim),
            nn.ReLU(),
            nn.Linear(gru_hidden_dim, cnn_channels) 
        )

        # MLP for final prediction
        self.MLP_out = nn.Sequential(
            nn.Linear(nodes + cnn_channels, gru_hidden_dim),
            nn.ReLU(),
            nn.Linear(gru_hidden_dim, 7) 
        )

        # Spatiotemporal module # 25 as hidden dim
        self.h_tz = Spatiotemporal(node_feature_dim = 10,#check
                                    edge_feature_dim = 32,
                                    hidden_dim = 25,
                                    gat_out_dim = 64,
                                    num_clusters = 3
                                    )
        
    def forward(self, x, spatio_out):
        # x: (batch_size, time_steps, num_nodes)

        # Apply CNN for feature extraction output shape: (batch_size, out_channels, time_steps)
        h_cnn = self.CNN_1D(x)   #eq 13

        # Reshape for GRU input (GRU expects (batch_size, time_steps, input_size))
        h_cnn = h_cnn.permute(0, 2, 1)  # Change shape to (batch_size, time_steps, out_channels)

        # Passing output of 1D CNN to GRU
        h_out, _ = self.gru(h_cnn)
        # eq 14

        # The GRU output is projected using MLP for temporal features
        h_out_tz = self.MLP(h_out[:, -1, :])  # Use last time step output from GRU
        # eq 15

        h_out_tz = h_out_tz.unsqueeze(1).repeat(1,16,1)
        
        # print("h_out_tz shape", h_out_tz.shape)
        # print("spatio out shape", spatio_out.shape)
        
        # Concatenate temporal and spatiotemporal features and pass through MLP_out for final prediction
        x = torch.cat([h_out_tz, spatio_out],dim=-1)

        
        final = self.MLP_out(x)  # Concatenate along the feature dimension
        # eq 16

        del h_cnn,h_out, h_out_tz, x
        return final

# SA_GNN

In [12]:
#node feature- PM2.5, windspeed, winddir, latitude, longitude

In [13]:
class SA_GNN(nn.Module):
    def __init__(self, num_stations, num_features, spatio_hidden, gat_out_dim,
                 temp_hidden, num_clusters, pred_len, time_steps, nodes):
        super(SA_GNN, self).__init__()
        
        self.hist_len = time_steps  # number of input time steps
        
        # Spatiotemporal Module
        self.spatio = Spatiotemporal(
            node_feature_dim = num_features,
            edge_feature_dim = 32,
            hidden_dim = spatio_hidden,
            gat_out_dim = gat_out_dim,
            num_clusters = num_clusters
        )
        # Temporal Module
        self.temporal = Temporal_Module(
            num_nodes = num_stations,
            cnn_channels = 64,
            kernel_size = 3,
            gru_hidden_dim = temp_hidden,
            future_step = pred_len,
            nodes = nodes
        )
        
        # Final output
        # self.fc = nn.Linear(temp_hidden + spatio_hidden, 1)
        self.fc = nn.Linear(nodes+7, 1)

    def forward(self, batch, device):
        
        # Spatiotemporal Features
        spatio_out = self.spatio(
            h_t = batch['features'][:, :self.hist_len], # (B, time_steps, num_features) # historical PM2.5
            v = batch['v'],                               # (B, N)
            D_ji = batch['D_ji'],                         # (N, N) or (B, N, N)
            adj = batch['adj'],                           # (B,T, N, N) 
            theta = batch['theta'],                       # (B, N)
            cluster_indices = batch['cluster_indices'],   # (B, N) or (N,)
            device = device
        )  # (B,T N)

      

        
        spatio_out = spatio_out.squeeze(-1)
        
        # print("spatio_out feature", spatio_out.shape)
        
        # Temporal Feature
        temporal_out = self.temporal(
            x = batch['pm25'].permute(0, 2, 1),  # (B, N, T)
            spatio_out = spatio_out
        )  # (B, pred_len, N)
#
        # print("temporal out shape",temporal_out.shape)
        
        x = torch.cat([temporal_out, spatio_out], dim=-1)
        # print("temporal and spatio_out concat shape",x.shape)

        
        final_out = self.fc(x)  

        del spatio_out, temporal_out, x
        return final_out

In [14]:
def generate_cluster_indices(df, n_clusters=3):
    df_cluster = df[['Station_ID', 'latitude', 'longitude', 'avg_pm25']].drop_duplicates()

    scaler = StandardScaler()
    features = scaler.fit_transform(df_cluster[['latitude', 'longitude', 'avg_pm25']])

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df_cluster['cluster_id'] = kmeans.fit_predict(features)

    return df_cluster[['Station_ID', 'cluster_id']]

# Distance matrix

In [15]:

def haversine_distance(lat1, lon1, lat2, lon2):

    R = 6371.0  # Radius of Earth in kilometers
    
    # Convert from degrees to radians
    lat1 = torch.deg2rad(lat1)
    lon1 = torch.deg2rad(lon1)
    lat2 = torch.deg2rad(lat2)
    lon2 = torch.deg2rad(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = torch.sin(dlat / 2)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon / 2)**2

    c = 2 * torch.arcsin(np.sqrt(a))
    
    distance = R * c
    
    del lat1, lat2, lon1, lon2, dlat, dlon, a, c
    return distance

In [16]:
def compute_distance_matrix(df, threshold=0.01):
    # latitudes = torch.tensor(df['Latitude'].values, dtype=torch.float32)
    # longitudes = torch.tensor(df['Longitude'].values, dtype=torch.float32)
    latitudes = torch.tensor(df['lat'].values, dtype=torch.float32)
    longitudes = torch.tensor(df['lon'].values, dtype=torch.float32)

    N = latitudes.shape[0]
    
    lat1 = latitudes.unsqueeze(0).repeat(N, 1)
    lon1 = longitudes.unsqueeze(0).repeat(N, 1)
    lat2 = latitudes.unsqueeze(1).repeat(1, N)
    lon2 = longitudes.unsqueeze(1).repeat(1, N)

    distance_matrix = haversine_distance(lat1, lon1, lat2, lon2)

    distance_matrix[distance_matrix < threshold] = 0  # Set small distances to 0

    
    del lat1, lat2, lon1, lon2
    return distance_matrix

# COMBINED DATA

In [17]:
df = pd.read_csv("/kaggle/input/delhi-dtaset/DelhiData_3hr_sample_Combine.csv")
# df = pd.read_excel("/kaggle/input/omklpol/COMBINED.xlsx")
# df.isna().sum()
df

,Time_Index,Station_ID,latitude,longitude,PM2.5,RH,SR,A2MTEMP,KINDEX,PBL,Precip,Surface_Pressure,U_Wind,V_Wind,WindX,WindY
0,0,0,28.815329,77.153010,288.371143,92.470159,11.286852,298.172481,35.430560,327.296569,0.001193,100929.9956,-1.978705,2.127121,0.301772,245.503435
1,0,1,28.646835,77.316032,308.531213,76.695000,12.750000,298.170967,35.663119,335.823994,0.001211,100915.7740,-1.691937,2.275459,0.300000,186.670000
2,0,2,28.695381,77.181665,145.786509,90.573100,12.631096,298.170967,35.663119,335.823994,0.001211,100915.7740,-1.691937,2.275459,0.350308,208.072876
3,0,3,28.470691,77.109936,213.198333,92.140000,45.279627,298.210318,36.038792,311.734019,0.000878,100936.6881,-2.352938,2.341858,0.291924,174.996658
4,0,4,28.776200,77.051074,250.166562,94.074341,12.059123,298.172481,35.430560,327.296569,0.001193,100929.9956,-1.978705,2.127121,0.284649,254.233259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332685,8754,33,28.550425,77.215938,142.666667,90.183333,23.108333,298.478205,0.689763,763.794124,0.000000,101446.9920,-3.964575,-5.832270,0.190000,275.180000
332686,8754,34,28.710508,77.249485,108.787454,87.308333,39.533333,298.721877,0.859711,772.747920,0.000000,101433.6070,-3.970310,-5.669805,1.950000,347.050000
332687,8754,35,28.531346,77.190156,114.354002,76.508333,47.775000,298.478205,0.689763,763.794124,0.000000,101446.9920,-3.964575,-5.832270,2.500000,19.700000
332688,8754,36,28.672342,77.315260,109.513465,90.525000,62.291667,298.721877,0.859711,772.747920,0.000000,101433.6070,-3.970310,-5.669805,1.200000,286.550000


In [18]:
print(len(df))
df = df[:5000]
len(df)

332690


5000

In [19]:
df.isna().sum()

Time_Index          0
Station_ID          0
latitude            0
longitude           0
PM2.5               0
RH                  0
SR                  0
A2MTEMP             0
KINDEX              0
PBL                 0
Precip              0
Surface_Pressure    0
U_Wind              0
V_Wind              0
WindX               0
WindY               0
dtype: int64

In [20]:
nodes = len(df)
nodes

5000

In [21]:
print(len(df.columns))
df.columns

16


Index(['Time_Index', 'Station_ID', 'latitude', 'longitude', 'PM2.5', 'RH',
       'SR', 'A2MTEMP', 'KINDEX', 'PBL', 'Precip', 'Surface_Pressure',
       'U_Wind', 'V_Wind', 'WindX', 'WindY'],
      dtype='object')

In [22]:
def create_adjacency_matrix(stations, sigma=1.5, threshold=0.01):
    N = len(stations)
    adj_matrix = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            dist = geodesic(
                (stations.iloc[i]['lat'], stations.iloc[i]['lon']),
                (stations.iloc[j]['lat'], stations.iloc[j]['lon'])
            ).km
            weight = np.exp(-dist**2 / (2 * sigma**2))
            if weight > threshold:
                adj_matrix[i, j] = weight
                
    return adj_matrix, stations

In [23]:
station = pd.read_csv("/kaggle/input/delhi-dtaset/stn_coord.csv")

adj_matrix, clustered_stations = create_adjacency_matrix(station)

print(adj_matrix)
len(adj_matrix)
print(station)
# print(clustered_stations)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.16911422 0.        ]
 [0.         0.         0.         ... 0.         0.         0.54287144]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.16911422 0.         ... 0.         0.         0.        ]
 [0.         0.         0.54287144 ... 0.         0.         0.        ]]
    id                            stn_name        lat        lon
0    0                              Alipur  28.815329  77.153010
1    1                         Anand.Vihar  28.646835  77.316032
2    2                         Ashok.Vihar  28.695381  77.181665
3    3                           Aya.Nagar  28.470691  77.109936
4    4                              Bawana  28.776200  77.051074
5    5                     Burari.Crossing  28.725650  77.201157
6    6                   CRRI.Mathura.Road  28.551201  77.273574
7    7      Dr..Karni.Singh.Shooting

In [24]:
distance_matrix = compute_distance_matrix(station)
# print(distance_matrix)

In [25]:
station.columns

Index(['id', 'stn_name', 'lat', 'lon'], dtype='object')

In [26]:
avg_pm25 = df.groupby('Station_ID')['PM2.5'].mean().reset_index()
avg_pm25.columns = ['Station_ID', 'avg_pm25']

station_loc = df[['Station_ID', 'latitude', 'longitude']].drop_duplicates()

station_features = pd.merge(station_loc, avg_pm25, on='Station_ID')

print(station_features)

    Station_ID   latitude  longitude    avg_pm25
0            0  28.815329  77.153010  176.144972
1            1  28.646835  77.316032  269.239617
2            2  28.695381  77.181665  106.203774
3            3  28.470691  77.109936  159.637988
4            4  28.776200  77.051074  192.014119
5            5  28.725650  77.201157  146.505966
6            6  28.551201  77.273574  192.389724
7            7  28.498571  77.264840  182.344121
8            8  28.750050  77.111261  299.313495
9            9  28.571027  77.071901  180.890329
10          10  28.655602  77.285932  179.346835
11          11  28.562776  77.118005  164.014746
12          12  28.681174  77.302523  139.792564
13          13  28.631694  77.249439  213.917352
14          14  28.732820  77.170633  143.270870
15          15  28.580280  77.233829  164.160104
16          16  28.591825  77.227307  183.756967
17          17  28.611281  77.237738   84.582781
18          18  28.636429  77.201067  221.326540
19          19  28.6

In [27]:

cluster_indices = generate_cluster_indices(station_features, n_clusters=3)
cluster_indices = torch.from_numpy(cluster_indices.values).long().to(device)
# len(cluster_indices)#station id cluster number

cluster_indices = cluster_indices[:,1]
print(cluster_indices)

tensor([1, 0, 2, 0, 1, 2, 0, 0, 1, 1, 2, 0, 2, 0, 2, 0, 0, 2, 0, 1, 1, 1, 0, 1,
        1, 0, 2, 0, 2, 2, 1, 1, 1, 0, 2, 0, 2, 2], device='cuda:0')


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [28]:
df = df.select_dtypes(include=[np.number])  # slectv only numeric

# Dataset

In [29]:
len(cluster_indices)

38

In [30]:
df

,Time_Index,Station_ID,latitude,longitude,PM2.5,RH,SR,A2MTEMP,KINDEX,PBL,Precip,Surface_Pressure,U_Wind,V_Wind,WindX,WindY
0,0,0,28.815329,77.153010,288.371143,92.470159,11.286852,298.172481,35.430560,327.296569,0.001193,100929.9956,-1.978705,2.127121,0.301772,245.503435
1,0,1,28.646835,77.316032,308.531213,76.695000,12.750000,298.170967,35.663119,335.823994,0.001211,100915.7740,-1.691937,2.275459,0.300000,186.670000
2,0,2,28.695381,77.181665,145.786509,90.573100,12.631096,298.170967,35.663119,335.823994,0.001211,100915.7740,-1.691937,2.275459,0.350308,208.072876
3,0,3,28.470691,77.109936,213.198333,92.140000,45.279627,298.210318,36.038792,311.734019,0.000878,100936.6881,-2.352938,2.341858,0.291924,174.996658
4,0,4,28.776200,77.051074,250.166562,94.074341,12.059123,298.172481,35.430560,327.296569,0.001193,100929.9956,-1.978705,2.127121,0.284649,254.233259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,131,17,28.611281,77.237738,95.694332,71.527509,213.750563,297.262873,-47.658740,1045.412327,0.000000,101138.3000,-1.498368,-6.222187,1.364165,148.729568
4996,131,18,28.636429,77.201067,91.783979,80.276667,230.750000,297.262873,-47.658740,1045.412327,0.000000,101138.3000,-1.498368,-6.222187,1.400000,134.170000
4997,131,19,28.684678,77.076574,173.246132,72.463620,229.296169,297.190226,-48.896075,1037.524459,0.000000,101130.7709,-1.333477,-6.087977,1.288500,122.449034
4998,131,20,28.570173,76.933762,190.804274,62.226993,218.013090,297.082768,-48.189452,1037.311273,0.000000,101134.9537,-1.068216,-6.195345,1.328510,158.931950


In [31]:
class DATASET_PM25(Dataset):
    def __init__(self, data, adj_matrix, distance_matrix,
                 wind_speed_column_index, wind_direction_column_index,
                 time_steps, future_step, cluster_indices):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.data = data
        self.adj_matrix = adj_matrix
        self.distance_matrix = distance_matrix.clone().detach().to(self.device).float()
        self.wind_speed_column_index   = wind_speed_column_index
        self.wind_direction_column_index = wind_direction_column_index
        self.time_steps   = time_steps
        self.future_step  = future_step
        self.cluster_indices = cluster_indices

        self.X, self.y = self.create_input_features(self.data,
            historical_steps=self.time_steps,
            prediction_length=self.future_step
        )

    def create_input_features(self, df, historical_steps=3, prediction_length=1):
        X = []
        y = []
        for t in range(historical_steps, len(df) - prediction_length):
            x_pmt = df.iloc[t-historical_steps:t][['latitude','longitude','PM2.5']].values
            x_feature = df.iloc[t-historical_steps:t][['U_Wind','V_Wind']].values
            y_t = df.iloc[t:t+prediction_length][['PM2.5']].values
            X.append((x_pmt, x_feature))   
            
            y.append(y_t)
            
        return X, y  

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_pmt, x_feature = self.X[idx]
        y = self.y[idx]
    
        N = len(self.cluster_indices)
        T = self.time_steps

        adj = self.adj_matrix
        features_batch = torch.zeros(T, N, 3, device=self.device)
        v_batch        = torch.zeros(T, N, device=self.device)
        theta_batch    = torch.zeros(T, N, device=self.device)
    
        for i, cluster_idx in enumerate(self.cluster_indices):
            pm25 = torch.tensor(x_pmt[:, 2], device=self.device)
            u = torch.tensor(x_feature[:, 0], device=self.device)
            v = torch.tensor(x_feature[:, 1], device=self.device)
    
            wind_speed     = torch.sqrt(u**2 + v**2)
            wind_dir_rad   = torch.atan2(v, u)
            wind_direction = torch.rad2deg(wind_dir_rad)
    
            features_batch[:, i, 0] = pm25
            features_batch[:, i, 1] = wind_speed
            features_batch[:, i, 2] = wind_direction
    
            v_batch[:, i]     = wind_speed
            theta_batch[:, i] = wind_direction

        return features_batch, adj, torch.tensor(y, dtype=torch.float, device=self.device), v_batch, theta_batch



In [32]:

import time
start_time = time.time()

dataset = DATASET_PM25(data = df, 
                adj_matrix = adj_matrix,
                 distance_matrix = distance_matrix, 
                 wind_speed_column_index = 12, 
                 wind_direction_column_index = 13,  
                 time_steps = 16, 
                 future_step = 8,
                 cluster_indices = cluster_indices)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, shuffle=True, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"DataLoader setup time: {time.time() - start_time} seconds")
print("Completed Dataset and Dataloader Creation.......................................................................")

DataLoader setup time: 47.79486417770386 seconds
Completed Dataset and Dataloader Creation.......................................................................


In [33]:
# for x, adj, y, v, theta in train_loader:
#     print(y)

In [34]:
torch.cuda.empty_cache()

# Model

In [35]:
model = SA_GNN(num_stations=38, num_features=3, spatio_hidden=25, gat_out_dim = 64, temp_hidden=32, 
               num_clusters=3, pred_len=8, time_steps = 16,  nodes=38).to(device)

In [36]:

learning_rate = 0.0015
batch_size_GRU_temporal = 32
batch_size_GRU_SA_GNN = 25
weight_decay = 0.0001
num_epoch = 50
hidden_units = 64

loss_fn = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

scheduler = StepLR(optimizer, step_size = 5, gamma = 0.5)

In [37]:
def L1_regularizer(model, lamda=0.01):
    l1_loss = 0
    for param in model.parameters():
        l1_loss += torch.sum(torch.abs(param))

    return lamda * l1_loss

In [38]:
def eval(y_true, y_pred):
    y_true = y_true.astype(np.float32)
    y_pred = y_pred.astype(np.float32)

    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)

    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0.0

    return mae, rmse, r2


In [39]:
# wind_speed_column_index = 13 
# wind_direction_column_index = 14
# pm25_column_index = 7

In [40]:
print("cluster_indices:", cluster_indices.shape, cluster_indices.dtype)
print("unique clusters:", torch.unique(cluster_indices))


cluster_indices: torch.Size([38]) torch.int64
unique clusters: tensor([0, 1, 2], device='cuda:0')


In [41]:
wind_speed_column_index = 12
wind_direction_column_index = 13
pm25_column_index = 4

In [42]:
pm25_data = []
for x, adj, y, v, theta in train_loader:
    pm25_data.append(x[:, :, pm25_column_index].cpu().numpy())

pm25_data = np.concatenate(pm25_data, axis=0)
mean = np.mean(pm25_data)
std = np.std(pm25_data)
del pm25_data

In [43]:
def train_loop(model, train_loader, optimizer, criterion, device, distance_matrix, cluster_indices, 
               wind_speed_column_index, wind_direction_column_index, pm25_column_index, mean, std):
    model.train()
    running_loss = 0.0

    for x, adj, y, v, theta in train_loader:
        x = x.to(device)
        adj = adj.to(device)
        y = y.to(device)
        v = v.to(device)
        theta = theta.to(device)

        # Only convert to dense if absolutely necessary
        adj = adj.to_dense() if adj.is_sparse else adj

        # Prepare batch
        batch = {
            'features': x,                          
            'adj': adj,                               
            'v': v, 
            'theta': theta, 
            'D_ji': distance_matrix,       
            'cluster_indices': cluster_indices, 
            'pm25': x[:, :, pm25_column_index]         
        }
        

        optimizer.zero_grad()
    
        output = model(batch, device)  # (B, pred_len, N)
        output = output.squeeze(-1)
        output = output[:, :8]

        # # rev standardization
        # output = output * std + mean
        # y = y * std + mean

        y = y.squeeze(-1)
        loss = criterion(output, y).to(device)  
        loss += L1_regularizer(model)  

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        del output, loss, x, adj, y, v, theta, batch
        torch.cuda.empty_cache()
        
    avg_loss = running_loss / len(train_loader)
    
    return avg_loss

In [44]:
pm25_data1 = []
for x, adj, y, v, theta in test_loader:
    pm25_data1.append(x[:, :, pm25_column_index].cpu().numpy())

pm25_data1 = np.concatenate(pm25_data1, axis=0)
mean1 = np.mean(pm25_data1)
std1 = np.std(pm25_data1)
del pm25_data1

In [45]:
def test_loop(model, test_loader, criterion, device,
              distance_matrix, cluster_indices, 
              wind_speed_column_index, wind_direction_column_index, pm25_column_index, mean1, std1):
    
    model.eval()
    running_loss = 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, adj, y, v , theta in test_loader:
            x = x.to(device)
            adj = adj.to(device)
            y = y.to(device)
            v = v.to(device)
            theta = theta.to(device)

            # Only convert to dense if absolutely necessary
            adj = adj.to_dense() if adj.is_sparse else adj

            batch = {
                'features': x,                          
                'adj': adj,                               
                'v': v, 
                'theta': theta, 
                'D_ji': distance_matrix,       
                'cluster_indices': cluster_indices, 
                'pm25': x[:, :, pm25_column_index]         
            }

            output = model(batch, device).to(device)  # (B, pred_len, N)
            output = output.squeeze(-1)
            output = output[ : , : 8]

            # output = output * std1 + mean1
            # y = y * std1 + mean1

            y = y.squeeze(-1)
            loss = criterion(output, y)
            running_loss += loss.item()

            all_preds.append(output.cpu())
            all_labels.append(y.cpu())

            del x, adj, v, theta, y, output, batch, loss

    avg_loss = running_loss / len(test_loader)
    
    preds = torch.cat(all_preds, dim=0).numpy()
    labels = torch.cat(all_labels, dim=0).numpy()

    mae, rmse ,r2 = eval(preds, labels)

    del preds, labels
    return avg_loss, mae, rmse, r2

# Trainning

In [46]:
torch.cuda.empty_cache()

In [ ]:
result=[]
mae_list = []
r2_list = []
rmse_list = []
train_loss_list = []
test_loss_list = []

print("Started Trainning...................")
for epoch in range(num_epoch):
    
    train_loss = train_loop(model, train_loader, optimizer, loss_fn, device, distance_matrix, cluster_indices, 
           wind_speed_column_index, wind_direction_column_index, pm25_column_index, mean, std)

    scheduler.step()
    test_loss, mae, rmse, r2 = test_loop(model, test_loader, loss_fn, device, distance_matrix, cluster_indices, 
          wind_speed_column_index, wind_direction_column_index, pm25_column_index, mean1, std1)

    mae_list.append(mae)
    rmse_list.append(rmse)
    r2_list.append(r2)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    
    print(f"Epoch {epoch+1}/{num_epoch} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f} | R2: {r2:.4f}")

    torch.cuda.empty_cache()
    del train_loss, test_loss, mae, rmse, r2
    
result.append(mae_list)
result.append(rmse_list)
result.append(r2_list)
result.append(train_loss_list)
result.append(test_loss_list)


Started Trainning...................
Epoch 1/50 | Train Loss: 6046.9805 | Test Loss: 5038.1122 | MAE: 54.0655 | RMSE: 70.8957 | R2: -3.0111
Epoch 2/50 | Train Loss: 4654.5927 | Test Loss: 4983.4204 | MAE: 49.7947 | RMSE: 70.4612 | R2: -2.7745
Epoch 3/50 | Train Loss: 4532.5754 | Test Loss: 5159.3713 | MAE: 56.0217 | RMSE: 71.7343 | R2: -4.4749
Epoch 4/50 | Train Loss: 4469.7765 | Test Loss: 4691.8644 | MAE: 48.9950 | RMSE: 68.3391 | R2: -2.6703
Epoch 5/50 | Train Loss: 4389.4127 | Test Loss: 4618.3925 | MAE: 49.0810 | RMSE: 67.7710 | R2: -2.2772


In [ ]:

fig, ax = plt.subplots(figsize=(8, 5))
epochs = np.arange(1, num_epoch + 1)

train = result[3] 
test = result[4]  


ax.plot(epochs, train, marker='o', label='Train loss', color = 'red')
ax.plot(epochs, test, marker='s', label='Test loss', color = 'grey')

ax.set_title("Train vs test loss plot", fontsize=14)
ax.set_xlabel("Epoch", fontsize=12)
ax.set_ylabel("Loss", fontsize=12)
ax.legend()
ax.grid(True)


# plt.tight_layout()
# plt.savefig("train_vs_test_loss.png", dpi=300)  
# plt.close()

In [ ]:
epochs = range(1, num_epoch + 1)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# MAE
axes[0].plot(epochs, result[0], marker='o', label='MAE')
axes[0].set_title("Mean Absolute Error")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("MAE")
axes[0].legend()
axes[0].grid(True)

# MRE
axes[1].plot(epochs, result[1], marker='o', label='RMSE')
axes[1].set_title("Root Mean Square Error")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("RMSE")
axes[1].legend()
axes[1].grid(True)

# RMSE
axes[2].plot(epochs, result[2], marker='o', label='R2')
axes[2].set_title("R square")
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("R2")
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()

# plt.savefig('plot_results_MAE_RMSE_R2.png')  
# plt.close()

# Predicted vs actual data

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
all_preds = []
all_labels = []
running_loss = 0
with torch.no_grad():
    for x, adj, y, v , theta in test_loader:
        x = x.to(device)
        adj = adj.to(device)
        y = y.to(device)
        v = v.to(device)
        theta = theta.to(device)

        # Only convert to dense if absolutely necessary
        adj = adj.to_dense() if adj.is_sparse else adj

        batch = {
            'features': x,                          
            'adj': adj,                               
            'v': v, 
            'theta': theta, 
            'D_ji': distance_matrix,       
            'cluster_indices': cluster_indices, 
            'pm25': x[:, :, pm25_column_index]         
        }

        output = model(batch, device)  # (B, pred_len, N)
        output = output.squeeze(-1)
        output = output[ : , : 8]

        # output = output * std1 + mean1
        # y = y * std1 + mean1

        y = y.squeeze(-1)
        loss = criterion(output, y)
        running_loss += loss.item()

        all_preds.append(output.cpu())
        all_labels.append(y.cpu())

avg_loss = running_loss / len(test_loader)

preds = torch.cat(all_preds, dim=0).numpy()
labels = torch.cat(all_labels, dim=0).numpy()

In [ ]:
# print(preds)
# print(labels)

In [ ]:
preds_sample = preds[4]
labels_sample = labels[4]

In [ ]:


plt.figure(figsize=(10, 5))
plt.plot(labels_sample, label="Actual", marker='o', linestyle="-")
plt.plot(preds_sample, label="Predicted", marker='s', linestyle="--")
plt.xlabel("Time Steps")
plt.ylabel("PM2.5")
plt.title("Prediction vs Actual PM2.5 concentration")
plt.legend()
plt.grid(True)

# plt.savefig("pm25_prediction_vs_actual.png", dpi=300, bbox_inches='tight')
# plt.close()